In [1]:
import requests
import json

In [2]:
def getUserToken(URL, user_data):
    data = requests.post(url, json=json.loads(user_data))
    authentication_data = json.loads(data.text)
    token = "Bearer " + str(authentication_data["result"])
    return token


def getProcessID(process_name, token):
    process_data = requests.get(f"https://platform.uipath.com/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
                                headers={"Authorization": token})
    process_json = json.loads(process_data.text)
    # print(json.dumps(process_json, indent=2))
    process_ID = process_json["value"][0]["Key"]
    return process_ID


def getRobotID(robotName, token):
    robot_name = requests.get(f"https://platform.uipath.com/odata/Robots?$filter=Name%20eq%20'{robotName}'",
                              headers={"Authorization": token})
    robot_json_obj = json.loads(robot_name.text)
    robot_ID = robot_json_obj["value"][0]["Id"]
    return robot_ID


def runJob(process_ID, robot_ID, token):
    start_job_json = """{ "startInfo":
       { "ReleaseKey": \"""" + process_ID + """\",
         "Strategy": "Specific",
         "RobotIds": [ """ + str(robot_ID) + """ ],
         "Source": "Manual",
         "InputArguments": "{'in_Arg1':'Aloha'}"        
       } 
    }"""  # InputArguments should be left {} or not included if workflow does not accept any input
    start_job_data = requests.post("https://platform.uipath.com/odata/Jobs/UiPath.Server.Configuration.OData.StartJobs",
                                   json=json.loads(start_job_json), headers={"Authorization": token})
    return start_job_data


에러 메시지 :  SSL: CERTIFICATE_VERIFY_FAILED , Certificate verify failed
인증서와 관련하여 유효하지 않기에 에러 메시지를 뱉고(?) Trackback과 함께 프로그램이 종료됩니다.

해결 방법

requests 모듈에서 url 뒤에 verify=False 옵션을 추가해 주면 해결할 수 있습니다.
해당 옵션을 사용하면 HTTPS 요청에 대한 SSL 인증서 확인 과정을 생략하겠다는 의미입니다.

response = requests.get(url,verify=False)

In [12]:
url="https://koreascoring.iptime.org/api/Account/Authenticate"
# tenancyName="Default"
# userName="admin"
# password="1q2w3e4r!"

user_data = {
	"tenancyName": "Default",
	"usernameOrEmailAddress": "admin",
	"password": "1q2w3e4r!"
}

headers={'Content-type':'application/json', 'Accept':'application/json'}

data = requests.post(url, json=user_data, headers=headers, verify=False)
authentication_data = json.loads(data.text)
print(authentication_data)
token = "Bearer " + str(authentication_data["result"])
print(token)

{'result': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjBERDBEODAzNDk3QTMwMkM1NUMwQjA0MzlBQjgzQzk4QkE4QjIwMDEiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJEZERZQTBsNk1DeFZ3TEJEbXJnOG1McUxJQUUifQ.eyJuYmYiOjE2MTg5OTQ0ODcsImV4cCI6MTYxODk5ODM4NywiaXNzIjoiaHR0cHM6Ly9rb3JlYXNjb3JpbmcuaXB0aW1lLm9yZy9pZGVudGl0eSIsImF1ZCI6Ik9yY2hlc3RyYXRvckFwaVVzZXJBY2Nlc3MiLCJjbGllbnRfaWQiOiI0ZDdmMmJmMi05MDkzLTQ5NTktOWQ1Mi1iNGE2NjBmMTQ4NWYiLCJzdWIiOiI2MTM4MGJhMy1iNjc1LTQ3NzUtYTEyOC05MTk4NzcyMmU0MjUiLCJhdXRoX3RpbWUiOjE2MTg5OTQ3ODcsImlkcCI6ImxvY2FsIiwiQXNwTmV0LklkZW50aXR5LlNlY3VyaXR5U3RhbXAiOiJET0tCUFZYQ0lDVlZZS0EyM1YzQlFGQkk1SUdDS1FGQiIsInBydF9pZCI6ImE1ZDk4NTBhLTE5NTgtNDYyMC05YWNhLTI5MjZlNzJlYTM4YSIsImhvc3QiOiJGYWxzZSIsImZpcnN0X25hbWUiOiIiLCJsYXN0X25hbWUiOiIiLCJwcnRfYWRtIjoiRmFsc2UiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhZG1pbiIsIm5hbWUiOiJhZG1pbiIsImVtYWlsIjoiYWRtaW5AZGVmYXVsdHRlbmFudC5jb20iLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInNjb3BlIjpbImVtYWlsIiwib3BlbmlkIiwicHJvZmlsZSIsInRlbmFudCIsIk9yY2hlc3RyYXRvckFwaVVzZXJBY2Nlc3MiXSwiYW1yIjpbInB3ZCJdfQ.ZUcxTEZFer

C:\Users\koreascoring\reactapp\backend\env\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'koreascoring.iptime.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
